In [1]:
# dependencies
import pypokedex
import requests
import json
import pandas as pd
import regex as re
import time

In [2]:
# pokemon moves url for requests
# limit=937 refers to the count of moves stored in the api
url = "https://pokeapi.co/api/v2/move/?limit=937"

# printing moves response object to console
response = requests.get(url)

In [18]:
# objective: create dataframe that includes all pokemon moves and their attributes

# create empty list to create dataframe with
move_details = []

data = response.json()
moves = data['results']

# iterate through api to add moves and attributes into the empty list
for move in moves:
    move_url = move['url']
    move_response = requests.get(move_url)
    move_data = move_response.json()
    move_id = re.search(r'/(\d+)/$', move_url).group(1)
    move_name = move_data['name']

    if 'effect_entries' in move_data and len(move_data['effect_entries']) > 0:
        move_effect = move_data['effect_entries'][0]['effect']
    else:
        move_effect = "Effect data not available."
    
    power = move_data['power']
    accuracy = move_data['accuracy']
    pp = move_data['pp']
    move_type = move_data['type']['name']
    damage_class = move_data['damage_class']['name']

    move_details.append({
        'Move ID': move_id,
        'Move Name': move_name,
        'Move Effect': move_effect,
        'Power': power,
        'Accuracy': accuracy,
        'PP': pp,
        'Type': move_type,
        'Damage Class': damage_class

    })

# save into dataframe
moves_df = pd.DataFrame(move_details)

# convert numerical columns into integers to match erd diagram
moves_df['Power'] = moves_df['Power'].astype('Int64')
moves_df['Accuracy'] = moves_df['Accuracy'].astype('Int64')
moves_df['PP'] = moves_df['PP'].astype('Int64')

# display dataframe
moves_df.head(5)

,Move ID,Move Name,Move Effect,Power,Accuracy,PP,Type,Damage Class
0,1,pound,Inflicts regular damage.,80,100,35,normal,physical
1,2,karate-chop,Inflicts regular damage. User's critical hit ...,80,100,25,fighting,physical
2,3,double-slap,Inflicts regular damage. Hits 2–5 times in on...,80,85,10,normal,physical
3,4,comet-punch,Inflicts regular damage. Hits 2–5 times in on...,80,85,15,normal,physical
4,5,mega-punch,Inflicts regular damage.,80,85,20,normal,physical


In [19]:
# verfify data types are correct
moves_df.dtypes

Move ID         object
Move Name       object
Move Effect     object
Power            Int64
Accuracy         Int64
PP               Int64
Type            object
Damage Class    object
dtype: object

In [20]:
   # export dataframe into csv file
moves_df.to_csv('moves_csv') 